# Pipeline ML via FastAPI - Demo complete

Ce notebook parcourt les **5 phases** du pipeline Data Science expose par l'API et visualise les resultats.

**Prerequis :** Lancer le serveur avant d'executer le notebook :
```bash
uv run uvicorn src.app.api:app --reload
```

In [1]:
import requests
import plotly.io as pio
import plotly.express as px
import pandas as pd
import json

BASE = "http://127.0.0.1:8000"

def show_plotly(artifact_json: str):
    """Affiche un artefact Plotly JSON en ligne."""
    fig = pio.from_json(artifact_json)
    fig.show()

def post(path, body=None):
    resp = requests.post(f"{BASE}{path}", json=body or {})
    resp.raise_for_status()
    return resp.json()

def get(path):
    resp = requests.get(f"{BASE}{path}")
    resp.raise_for_status()
    return resp.json()

## Phase 1 - Nettoyage des donnees

**Objectif :** Generer un jeu de donnees "sale" (valeurs manquantes, doublons, outliers), puis appliquer un pipeline de nettoyage automatique.

On observe le rapport qualite **avant** et **apres** nettoyage pour verifier que :
- Les valeurs manquantes ont ete imputees
- Les doublons ont ete supprimes
- Les variables categoriques ont ete encodees (one-hot encoding)

In [2]:
# Generation d'un dataset avec 200 observations, seed fixe pour reproductibilite
data = post("/dataset/generate", {"phase": "clean", "seed": 1, "n": 200})
ds_clean_id = data["meta"]["dataset_id"]
print(f"Dataset ID: {ds_clean_id}")

# Rapport qualite AVANT nettoyage : on observe les valeurs manquantes, doublons, types
report = get(f"/clean/report/{ds_clean_id}")
print("\nRapport qualite (avant nettoyage) :")
pd.DataFrame(report["result"]["data"]).T

Dataset ID: 597b0c63-7289-423c-a90a-03396ef95c7c

Rapport qualite (avant nettoyage) :


,x1,x2,x3,segment,target
rows,206,206,206,206,206
columns,5,5,5,5,5
missing_values,38,21,0,21,0
total_missing,80,80,80,80,80
duplicates,6,6,6,6,6
dtypes,float64,float64,float64,str,int64


In [3]:
# Ajustement du cleaner (imputation moyenne par defaut, one-hot encoding)
fit_resp = post("/clean/fit", {"dataset_id": ds_clean_id, "params": {}})
cleaner_id = fit_resp["result"]["cleaner_id"]
print(f"Cleaner ID: {cleaner_id}")

# Transformation : applique le pipeline sur le dataset
transform_resp = post("/clean/transform", {"dataset_id": ds_clean_id, "cleaner_id": cleaner_id})
clean_ds_id = transform_resp["meta"]["dataset_id"]

# Comparaison avant/apres
quality_before = fit_resp["result"]["metrics"]["quality_before"]
quality_after = transform_resp["result"]["metrics"]["quality_after"]

print(f"Avant : {quality_before['total_missing']} valeurs manquantes, {quality_before['duplicates']} doublons")
print(f"Apres : {quality_after['total_missing']} valeurs manquantes, {quality_after['duplicates']} doublons")
print(f"Colonnes : {quality_before['columns']} -> {quality_after['columns']} (one-hot encoding des categoriques)")

# Apercu des donnees nettoyees
print("\nApercu des donnees nettoyees :")
pd.DataFrame(transform_resp["result"]["data"]).head()

Cleaner ID: b3c11e2f-8464-4ac6-83fe-4d78a17076a0
Avant : 80 valeurs manquantes, 6 doublons
Apres : 0 valeurs manquantes, 0 doublons
Colonnes : 5 -> 7 (one-hot encoding des categoriques)

Apercu des donnees nettoyees :


,x1,x2,x3,target,segment_A,segment_B,segment_C
0,124.365180,0.490623,1.0,1.0,0.0,0.0,1.0
1,90.823654,0.322681,38.0,1.0,0.0,1.0,0.0
2,92.077424,0.670789,73.0,1.0,0.0,0.0,1.0
3,108.165139,0.450874,44.0,1.0,1.0,0.0,0.0
4,112.981114,0.382103,25.0,0.0,0.0,0.0,1.0


## Phase 2 - Analyse exploratoire (EDA)

**Objectif :** Comprendre la distribution des variables et identifier des tendances.

L'API genere automatiquement :
- **Histogrammes** pour chaque variable numerique (distribution)
- **Bar charts** pour les variables categoriques (comptage)
- **Boxplots** numerique vs categorique (comparaison entre groupes)

In [4]:
# Generation d'un dataset EDA (age, income, spend, visits, segment, channel)
data = post("/dataset/generate", {"phase": "eda", "seed": 2, "n": 200})
ds_eda_id = data["meta"]["dataset_id"]

# Statistiques descriptives
summary = post("/eda/summary", {"dataset_id": ds_eda_id})

print("Statistiques numeriques :")
print("  -> On observe les moyennes, ecarts-types, min/max et taux de valeurs manquantes")
pd.DataFrame(summary["result"]["data"]["numeric"]).T

Statistiques numeriques :
  -> On observe les moyennes, ecarts-types, min/max et taux de valeurs manquantes


,count,mean,std,min,25%,50%,75%,max,missing_rate
age,200.0,45.480000,14.665777,18.000000,33.750000,46.000000,58.000000,69.000000,0.00
income,200.0,53204.605858,31528.958319,12418.128019,39525.863965,48808.362957,59967.494443,261279.472738,0.00
spend,196.0,1024.040189,1062.166190,3.341020,321.449853,683.815031,1353.227194,6504.466144,0.02
visits,200.0,4.800000,2.178008,0.000000,3.000000,5.000000,6.000000,12.000000,0.00


In [5]:
# Generation de tous les graphiques EDA (Plotly interactif)
plots_resp = post("/eda/plots", {"dataset_id": ds_eda_id})
artifacts = plots_resp["artifacts"]

print(f"{len(artifacts)} graphiques generes : {list(artifacts.keys())}\n")

for name, plot_json in artifacts.items():
    show_plotly(plot_json)

# Interpretation :
# - Les histogrammes montrent la forme de la distribution (normale, asymetrique, bimodale)
# - Les bar charts revelent les desequilibres entre categories
# - Les boxplots permettent de comparer les distributions entre segments
#   et d'identifier d'eventuels outliers (points au-dela des moustaches)

10 graphiques generes : ['hist_age', 'hist_income', 'hist_spend', 'hist_visits', 'bar_segment', 'bar_channel', 'box_age_by_segment', 'box_income_by_segment', 'box_spend_by_segment', 'box_visits_by_segment']



## Phase 3 - Analyse multivariee

**Objectif :** Explorer les relations entre variables via l'ACP et le clustering.

- **ACP (PCA)** : Reduit les dimensions tout en conservant un maximum de variance. La variance expliquee indique la qualite de la projection.
- **KMeans** : Regroupe les observations en clusters. Le score silhouette (entre -1 et 1) mesure la coherence des clusters : plus il est proche de 1, mieux les clusters sont separes.

In [6]:
# ACP : projection en 2 dimensions
data = post("/dataset/generate", {"phase": "mv", "seed": 3, "n": 200})
ds_mv_id = data["meta"]["dataset_id"]

pca_resp = post("/mv/pca/fit_transform", {"dataset_id": ds_mv_id, "n_components": 2})
pca_data = pca_resp["result"]["data"]

var_ratio = pca_data["explained_variance_ratio"]
print(f"Variance expliquee : PC1={var_ratio[0]:.1%}, PC2={var_ratio[1]:.1%}")
print(f"Total : {sum(var_ratio):.1%} de la variance conservee sur 2 axes")

# Scatter plot de la projection ACP
projected = pd.DataFrame(pca_data["projected_data"], columns=["PC1", "PC2"])
fig = px.scatter(projected, x="PC1", y="PC2", title="Projection ACP (2D)",
                 labels={"PC1": f"PC1 ({var_ratio[0]:.1%})", "PC2": f"PC2 ({var_ratio[1]:.1%})"})
fig.show()

Variance expliquee : PC1=41.1%, PC2=19.0%
Total : 60.1% de la variance conservee sur 2 axes


In [7]:
# KMeans : clustering en 3 groupes
kmeans_resp = post("/mv/cluster/kmeans", {"dataset_id": ds_mv_id, "n_clusters": 3})
kmeans_data = kmeans_resp["result"]["data"]

sil_score = kmeans_data["silhouette_score"]
print(f"Score silhouette : {sil_score:.3f}")
if sil_score > 0.5:
    print("  -> Bonne separation des clusters")
elif sil_score > 0.25:
    print("  -> Separation moderee, les clusters se chevauchent partiellement")
else:
    print("  -> Faible separation, les clusters sont peu distincts")

# Visualisation des clusters sur la projection ACP
projected["cluster"] = [str(c) for c in kmeans_data["labels"]]
fig = px.scatter(projected, x="PC1", y="PC2", color="cluster",
                 title=f"Clusters KMeans (k=3) sur ACP — Silhouette={sil_score:.3f}",
                 labels={"cluster": "Cluster"})
fig.show()

Score silhouette : 0.361
  -> Separation moderee, les clusters se chevauchent partiellement


In [8]:
# Matrice de correlation : identifie les relations lineaires entre variables
# Valeurs proches de +1/-1 = forte correlation, proches de 0 = pas de relation lineaire
mv_report = get(f"/mv/report/{ds_mv_id}")
corr = mv_report["result"]["data"]["correlation_matrix"]

corr_df = pd.DataFrame(corr)
fig = px.imshow(corr_df, text_auto=".2f", title="Matrice de correlation",
                color_continuous_scale="RdBu_r", zmin=-1, zmax=1)
fig.show()

## Phase 4 - Modeles ML de base

**Objectif :** Entrainer et comparer deux modeles de classification sur le meme jeu de donnees.

- **Regression Logistique** : modele lineaire, rapide et interpretable
- **Random Forest** : ensemble d'arbres de decision, capture les non-linearites

Metriques evaluees :
- **Accuracy** : proportion de bonnes predictions
- **Precision/Recall** : compromis entre faux positifs et faux negatifs
- **F1** : moyenne harmonique precision/recall
- **AUC** : capacite du modele a discriminer les classes (1 = parfait, 0.5 = aleatoire)

In [9]:
# Generation du dataset ML (300 obs pour un split train/test plus robuste)
data = post("/dataset/generate", {"phase": "ml", "seed": 4, "n": 300})
ds_ml_id = data["meta"]["dataset_id"]

# Entrainement : Regression Logistique
logreg = post("/ml/train", {"dataset_id": ds_ml_id, "target_col": "target", "model_type": "logreg"})
logreg_id = logreg["result"]["model_id"]
print("Regression Logistique :")
pd.Series(logreg["result"]["metrics"]).map(lambda x: f"{x:.3f}")

Regression Logistique :


accuracy     0.783
precision    0.787
recall       0.783
f1           0.783
auc          0.901
dtype: str

In [10]:
# Entrainement : Random Forest
rf = post("/ml/train", {"dataset_id": ds_ml_id, "target_col": "target", "model_type": "rf"})
rf_id = rf["result"]["model_id"]
print("Random Forest :")
pd.Series(rf["result"]["metrics"]).map(lambda x: f"{x:.3f}")

Random Forest :


accuracy     0.817
precision    0.820
recall       0.817
f1           0.817
auc          0.894
dtype: str

In [11]:
# Comparaison visuelle des deux modeles
comparison = pd.DataFrame({
    "Regression Logistique": logreg["result"]["metrics"],
    "Random Forest": rf["result"]["metrics"]
})
fig = px.bar(comparison.T, barmode="group", title="Comparaison des modeles de base",
             labels={"value": "Score", "index": "Modele"})
fig.update_layout(yaxis_range=[0, 1])
fig.show()

# Interpretation :
# - Si les scores sont proches, le probleme est probablement lineairement separable
# - Si Random Forest domine nettement, il y a des relations non-lineaires
# - L'AUC est souvent la metrique la plus fiable pour comparer des classifieurs

## Phase 5 - ML avance (Tuning et Explicabilite)

**Objectif :** Optimiser les hyperparametres et comprendre les predictions du modele.

- **GridSearchCV** : teste toutes les combinaisons d'hyperparametres avec validation croisee
- **Feature importance globale** : quelles variables influencent le plus le modele dans son ensemble
- **Importance par permutation** : mesure la baisse de performance quand on melange aleatoirement une variable
- **Explication d'instance** : pourquoi le modele a fait cette prediction pour un individu precis

In [12]:
# Tuning : GridSearchCV sur un Random Forest
# On teste 3x3 = 9 combinaisons de (n_estimators, max_depth)
data = post("/dataset/generate", {"phase": "ml_advanced", "seed": 5, "n": 300})
ds_ml2_id = data["meta"]["dataset_id"]

tune_resp = post("/ml2/tune", {
    "dataset_id": ds_ml2_id,
    "target_col": "target",
    "model_type": "rf",
    "param_grid": {"n_estimators": [5, 10, 50], "max_depth": [3, 5, 10]}
})
best_model_id = tune_resp["result"]["model_id"]

print(f"Meilleurs hyperparametres : {tune_resp['result']['data']['best_params']}")
print(f"Score CV (validation croisee) : {tune_resp['result']['metrics']['best_cv_score']:.3f}")
print(f"Score test : {tune_resp['result']['metrics']['test_score']:.3f}")

# Si score CV >> score test : risque de surapprentissage
# Si score CV ~ score test : le modele generalise bien
diff = abs(tune_resp['result']['metrics']['best_cv_score'] - tune_resp['result']['metrics']['test_score'])
if diff < 0.05:
    print("  -> Le modele generalise bien (ecart CV/test < 5%)")
else:
    print(f"  -> Attention : ecart CV/test de {diff:.1%}, possible surapprentissage")

Meilleurs hyperparametres : {'max_depth': 3, 'n_estimators': 50}
Score CV (validation croisee) : 0.917
Score test : 0.933
  -> Le modele generalise bien (ecart CV/test < 5%)


In [13]:
# Importance globale des features (basee sur la reduction d'impurete dans les arbres)
fi_resp = get(f"/ml2/feature-importance/{best_model_id}")
fi = pd.Series(fi_resp["result"]["data"]).sort_values(ascending=True)

fig = px.bar(x=fi.values, y=fi.index, orientation="h",
             title="Importance des features (globale - impurete)",
             labels={"x": "Importance", "y": "Feature"})
fig.show()

# Les features en haut du graphique sont les plus influentes pour le modele
print(f"Feature la plus importante : {fi.index[-1]} ({fi.values[-1]:.3f})")
print(f"Feature la moins importante : {fi.index[0]} ({fi.values[0]:.3f})")

Feature la plus importante : f11 (0.245)
Feature la moins importante : f1 (0.006)


In [14]:
# Importance par permutation : plus robuste que l'importance par impurete
# On melange aleatoirement chaque variable et on mesure la baisse de performance
# Une forte baisse = la variable est vraiment utile au modele
perm_resp = post("/ml2/permutation-importance", {
    "model_id": best_model_id,
    "dataset_id": ds_ml2_id,
    "target_col": "target"
})
perm = pd.Series(perm_resp["result"]["data"]).sort_values(ascending=True)

fig = px.bar(x=perm.values, y=perm.index, orientation="h",
             title="Importance par permutation",
             labels={"x": "Baisse de performance", "y": "Feature"})
fig.show()

# Comparaison avec l'importance globale :
# Si les deux classements concordent, on est confiant sur les features cles
# Des divergences peuvent indiquer des correlations entre features

In [15]:
# Explication d'une instance individuelle
# On cree un individu fictif et on demande au modele pourquoi il predit cette classe
model_info = get(f"/ml/model-info/{best_model_id}")
features = model_info["result"]["data"]["features"]
instance = {f: 0.5 for f in features}

explain_resp = post("/ml2/explain-instance", {
    "model_id": best_model_id,
    "instance": instance
})
contributions = explain_resp["result"]["data"]
print(f"Prediction pour cet individu : classe {contributions.get('prediction')}")

if "contributions" in contributions:
    contribs = pd.Series(contributions["contributions"]).sort_values()
    fig = px.bar(x=contribs.values, y=contribs.index, orientation="h",
                 title="Contributions des features a la prediction",
                 labels={"x": "Contribution", "y": "Feature"},
                 color=contribs.values,
                 color_continuous_scale="RdBu_r")
    fig.show()
    
    # Interpretation :
    # - Contributions positives : poussent vers la classe predite
    # - Contributions negatives : poussent vers l'autre classe
    print("\nFeatures qui poussent le plus vers cette prediction :")
    top = contribs.tail(3)
    for feat, val in top.items():
        print(f"  {feat}: {val:+.4f}")

Prediction pour cet individu : classe 0
